In [0]:
!pip3 install pymagnitude
# !wget http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude

from pymagnitude import *

In [0]:
import numpy as np
import torch
import string
import nltk, string, os, json

from nltk import RegexpParser
from nltk.tree import Tree
import random
import ipywidgets as widgets

from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


##Embeddings

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [0]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# def find_similar_command(user_command, known_commands):

#   sim1 = find_most_similar_command(user_command, known_commands)
#   sim = []
#   for k in known_commands:
#     sim.append(cosine(model.encode(user_command)[0], model.encode(k)[0]))
#   # sim = [float(i)/sum(sim) for i in sim]
#   for i in range(len(sim)):
#     sim[i] += sim1[i]
#   idx = sim.index(max(sim))
#   #print(sim)
#   #print(idx)
#   return known_commands[idx]

def find_similar_command(user_command, known_commands):
  # TODO - Do something
  known_commands_embeddings = embed(known_commands)
  known_commands_embeddings = np.array(known_commands_embeddings).tolist()
  user_command_embedding = embed([user_command])
  user_command_embedding = np.array(user_command_embedding).tolist()[0]

  sim = []
  for k in known_commands_embeddings:
    sim.append(cosine(user_command_embedding, k))
  idx = sim.index(max(sim))
  return known_commands[idx]
  # user_sent = construct_sentence_vector(user_command)
  # known_sent = []
  # for command in known_commands:
  #   known_sent.append(construct_sentence_vector(command))
  # #ans = vectors.similarity(user_sent, known_sent) 
  # known_sent = np.array(known_sent)
  # sim = vectors.similarity(user_sent, known_sent).tolist()
  # # ind = sim.index(max(sim))
  # # print(sim)
  # # return known_commands[ind]
  # return sim

##Parse Json

In [0]:
def get_article(filename):
  with open(filename, "r") as f:
    s = [] # steps - list of tuples (method_number, step text)
    d = {} # descriptions - Description dictionary step text -> description
    m = [] # methods
    data_json = json.load(f)
    title = data_json['title']
    title_description = data_json['title_description']
    category_list = []
    # Get the categories
    if data_json['category_hierarchy']:
      category_list = data_json['category_hierarchy']
    if title != None:
    # Get the steps
        if len(data_json['method/part']) > 0:
            methods = data_json['method/part']
            for method in methods:
                all_steps = [(method['name'], step['headline']) for step in method['steps']]
                s += all_steps
                m += method['name']
                all_descs = dict(zip([step['headline'] for step in method['steps']], [step['description'] for step in method['steps']]))
                d.update(all_descs)      
        else:
          # Steps could be separate from methods
          if data_json['steps']:
            for step in data_json['steps']:
              s.append(('', step['headline']))
              d[step['headline']] = step['description']
            # s = [(None, step[0]) for step in data_json['steps']]
            # d = dict(zip([step[0] for step in data_json['steps']], [step[1] for step in data_json['steps']]))
    else:
        print('Article has no title!')
    return s, d, m, title, category_list, title_description

##Imperative Detection

In [0]:
def is_imperative(tagged_sent):
    if tagged_sent[-1][0] != "?":
         # Catch simple imperatives
        if tagged_sent[0][1] == "VB" or tagged_sent[0][1] == "MD":
            return True
        elif tagged_sent[0][0].lower() == "don\'t":
          # If the first token is don't, then it is imperative as well
          return True
        else:
          possible_verb = ''
          # determine which word to check, if first word is an adverb check the second one
          if tagged_sent[0][1] == "RB":
            possible_verb = tagged_sent[1][0]
          else:
            possible_verb = tagged_sent[0][0]
          # check if the first word has a verb synonym
          synonyms = [syns.name() for syns in wordnet.synsets(possible_verb)]
          # split the synonyms array into two: more related synonyms should appear earlier
          # Follow a heuristic such that if a verb synonym doesn't appear in the first half
          # then this word is likely more often used as a noun
          synonyms = synonyms[0:len(synonyms)//2]
          for synonym in synonyms:
            # if there is a verb written the same way as the first token, return true
            splitted = synonym.split('.')
            if splitted[1] == 'v':
              if splitted[0] == possible_verb.lower():
                return True
    # For the sake of our game, consider the questions as non-imperative  
    return False

# Tokenizer
def tokenize(text):
    tokens = []
    words = text.split(' ')
    for word in words:
        if len(word) == 0:
            continue
        w = []
        for c in word:
            if c in string.punctuation:
              if c == '\'':
                w.append(c)
              else:
                  if len(w) != 0:
                      tokens.append(''.join(w))
                      w = []
                  tokens.append(c)
            else:
                if c not in string.whitespace:
                    w.append(c)
        if len(w) != 0:
            tokens.append(''.join(w))
    return tokens

def imperative_check(text):
  # First tokenize the step headline
  tokens = tokenize(text)
  # Get the POS-tags
  tags = nltk.pos_tag(tokens)
  # Check if imperative
  imperative = is_imperative(tags)
  
  return imperative

## Change to third person

In [0]:
# refered from https://stackoverflow.com/questions/41051125/turning-a-sentence-from-first-to-second-person

forms = {"am" : "are", 'i' : 'you', 'my' : 'your', 'me' : 'you', 'mine' : 'yours'} #'you' : 'I', 'your' : 'my', 'yours' : 'mine', "are" : "am"}

def translate(text):
  # print(text)
  if '"' in text:
    result = ""
    _text = text.split('"')
    l = len(_text)
    for i in range(l):
      if i%2==0:
        result += _text[i]
        for word in _text[i].split():
          # print(word)
          if word.lower() in forms: 
            result = result.replace(word, forms[word.lower()])
      else:
        s = '"' + _text[i] + '"'
        result+= s
    return result

  result = text
  for word in text.split():
    # print(word)
    if word.lower() in forms: 
      result = result.replace(word, forms[word.lower()])
      # print(result)
  # print(result)
  return result

def changePerson(text):
  result = ""
  sentences = text.split(".")
  # print(text)
  for _s in sentences:
    s = _s.strip()
    if s=="":
      continue
    tags = nltk.pos_tag(tokenize(s))
    isImp = is_imperative(tags)

    r = ""
    if isImp:
      if s[0] == ' ' or s[0] == '\n':
        r += "You " + s[1].lower() + s[2:]
      else:
        r += "You " + s[0].lower() + s[1:]
    else:
      r = s
    result += translate(r)
    result+='.\n'
  return result

def identifyObjects(text):
  tags = nltk.pos_tag(tokenize(text))
  objects = []
  for t in tags:
    if t[1]=="NN":
      objects.append(t[0])
  return objects


## Description Generator

In [0]:
def get_description_title(title):
  tit=""
  title = title.lower()
  if len(title) == 0:
    return
  if title.split()[0].lower()=="how" and title.split()[1].lower()=="to":
    tit += "You want "
    tit += " ".join(title.split()[1:])
  print(tit)

def get_description_title_description(title_description):
  if title_description is None or title_description=="":
    return
  print(changePerson(title_description))

def get_step_description_game(step, description):
  des = description[step]
  # print(des)
  if(des==""):
    return
  d = changePerson(des)
  return d


##Wrong Choices/Similar Steps

In [0]:
def find_steps_similar_articles(title, categories=[]):
  vectors = Magnitude("drive/Shared drives/CIS 700 Project/uniemb.magnitude")
  # v=construct_sentence_vector(title, vectors)
  sen_vec = embed([title])
  v = np.array(sen_vec).tolist()[0]
  sim = vectors.most_similar(v, topn = 10)
  steps = []
  cnt=0
  
  for s in sim:
    f = open('drive/Shared drives/CIS 700 Project/WikihowData/js_files/' + s[0] + '.json')
    data_json = json.load(f) 
    
    tit = data_json['title']
    cats = data_json['category_hierarchy']
    if categories==None or len(categories)==0 or len(list(set(cats) & set(categories)))>0:
      if tit.lower()!=title.lower() and cnt<3:
        cnt+=1
        if len(data_json['method/part']) > 0:
          methods = data_json['method/part']
          for method in methods:
              all_steps = [step['headline'] for step in method['steps']]
              steps += all_steps 
        else:
          if data_json['steps']:
            for step in data_json['steps']:
              steps.append(step['headline'])       
  return steps


## Subcategories

In [0]:
#Stored the sub categories data in categories.json
#just load the data for categories 
with open('drive/Shared drives/CIS 700 Project/categories.json', 'r') as f:
  sub_categories = json.load(f)

In [0]:
for k in sub_categories.keys():
  print(k)

In [0]:
total_files = 0
for k in sub_categories.keys():
  total_files += len(sub_categories[k])
print(total_files)

11225


## Gameplay functions

In [0]:
def trim_beginning_punctuation(sentence):
  to_return = ''
  letter_seen = False
  for c in sentence:
    if c in string.punctuation or c in string.whitespace:
      if letter_seen:
        to_return += c
    else:
      if c in string.ascii_letters or c in string.digits:
        if not letter_seen:
          letter_seen = True
        to_return += c
  return to_return

def milestone_1_results(filename, include_desc):
  # include_desc -> Do you want to include imperative description sentences as game steps?
  steps, descriptions, method_names, title, category_list, title_dis = get_article(filename)
  # Once the necessary fields are filled, find imperatives
  imperatives = []

  for step in steps:
    # First split the headline into sentences
    headline_sentences = step[1].split('.')
    if len(headline_sentences) == 1:
      headline_sentences = headline_sentences[0].split(':')
    for headline in headline_sentences:
      trimmed_headline = trim_beginning_punctuation(headline)
      trimmed_headline = trimmed_headline.replace('\n', '')
      if len(trimmed_headline) == 0:
          continue
      if imperative_check(trimmed_headline):
        if trimmed_headline not in imperatives:
          imperatives.append(trimmed_headline)
          # fix the way it is represented in descriptions
          old_desc = descriptions[step[1]]
          del descriptions[step[1]]
          descriptions[trimmed_headline] = old_desc
        break # No two imperatives from the same headline are added.
    if include_desc:
      description = descriptions[step[1]]
      # Split the description paragraph into sentences
      sentences = description.split('.')
      for sentence in sentences:
        trimmed_sentence = trim_beginning_punctuation(sentence)
        trimmed_sentence = trimmed_sentence.replace('\n', '')
        if len(trimmed_sentence) == 0:
          continue
        if imperative_check(trimmed_sentence):
          if trimmed_sentence not in imperatives:
            imperatives.append(trimmed_sentence)
          break # No two imperatives from the same description are added. !!! REMOVE THIS IF you want multiple imperatives
    
  #third_person = [changePerson(imperative) for imperative in imperatives]
  return imperatives, title, descriptions, category_list, title_dis

Choice picking

In [0]:
def pick_choices(current_step, commands):
  if len(commands) - 1 == current_step:
    return None, None
  elif len(commands) - 2 == current_step:
    # Can only pick one wrong choice
    return len(commands) - 1, None
  else:
    # pick 2 that are not in steps_seen
    options_first = [num for num in range(current_step + 1, len(commands))]
    first = random.choice(options_first)
    options_second = [num for num in range(current_step + 1, len(commands)) if num != first]
    second = random.choice(options_second)
    return first,second

def pick_choices_alt(title, n, similars, curr_step, further_steps):
  # first filter the ones that are too similar to actual correct step
  user_sent = embed([curr_step])
  user_sent = np.array(user_sent).tolist()[0]
  # user_sent = construct_sentence_vector(curr_step)
  similar_steps = embed(similars)
  similar_steps = np.array(similar_steps).tolist()

  s = []
  for idx, k in enumerate(similar_steps):
    a = cosine(user_sent, k)
    if a<0.9 and a>0.1:
      s.append(similars[idx])
  # s = [sim for sim in similars if vectors.similarity(construct_sentence_vector(sim), user_sent) < 0.9 and vectors.similarity(construct_sentence_vector(sim), user_sent) > 0.1]
  s = s + further_steps
  first = random.choice(s)
  s_2 = [step for step in s if step != first]
  second = random.choice(s_2)
  return first,second

In [0]:
wrong_warning = ['Whoa! You should not do that yet.', 'Umm.. maybe you should not do that now.', 'That could work in an ideal world, but that is not what we have here. Try again please!', 'No no no...']

##Game Loop

In [0]:
current_step = 0
steps_seen = []
end_game = False
file_name = ''
# First get the game
print('Welcome to WikiHow Dungeon!')
print('You can either pick a category to play a game from or you can upload your own game.')
print('a) Pick a category')
print('b) Upload JSON')
picked = ''
is_picked = False
while not is_picked:
  picked = input('>')
  if picked.lower() == 'a' or picked.lower() == 'b':
    is_picked = True
  else:
    print('Try again.')
# If from category
if picked.lower() == 'a':
  print('Enter the corresponding letter for a category below:')
  print('a) School: university, other school stuff')
  print('b) Youth: dating, culture, interactions, social events')
  print('c) Work: going to work, socializing at work')
  print('d) Animals: dogs, cats, birds')
  print('e) Fun: activities, parties, toys, music, drinks, games, halloween')
  print('f) Home and Family: parents, weddings, neighbors')
  print('g) Pop Culture: fandom, celebrities')
  print('h) Relationships: single life, dating, relationship issues')
  print('i) Friendship: friends, traveling with companions, nicknames')
  print('k) Appearance: fashion, looking good')
  is_picked = False
  while not is_picked:
    picked = input('>')
    if len(picked) == 1 and picked.lower() in 'abcdefghik':
      is_picked = True
    else:
      print('Try again.')
  categories = []
  if picked.lower() == 'a':
    categories = ['University', 'School Stuff']
  elif picked.lower() == 'b':
    categories = ['Youth Dating', 'Youth Culture', 'Social Interactions for Youth', 'Social Events for Youth']
  elif picked.lower() == 'c':
    categories = ['Job Attendance', 'Socializing at Work', 'School Stuff']
  elif picked.lower() == 'd':
    categories = ['Dogs', 'Cats', 'Birds']
  elif picked.lower() == 'e':
    categories = ['Drinks', 'Games', 'Parties', 'Fun Activities', 'Toys', 'Halloween', 'Music']
  elif picked.lower() == 'f':
    categories = ['Youth and Family', 'Parents', 'School Stuff', 'Weddings', 'Neighbors']
  elif picked.lower() == 'g':
    categories = ['Celebrities', 'Fandom']
  elif picked.lower() == 'h':
    categories = ['Relationship Issues', 'Single Life', 'Dating']
  elif picked.lower() == 'i':
    categories = ['Friends', 'Social Interactions', 'Traveling with Companions', 'Nicknames']
  elif picked.lower() == 'k':
    categories = ['Fashion', 'Looking Good']
  c = random.choice(categories)
  articles = sub_categories[c]
  found_imperatives = False
  while not found_imperatives:
    file_name = random.choice(articles)
    file_name = "/content/drive/Shared drives/CIS 700 Project/category_files/" + file_name
    # Get the commands, descriptions and title
    known_commands, title, descriptions, category_list, title_description = milestone_1_results(file_name, False)
    if len(known_commands) > 0:
      found_imperatives = True

elif picked.lower() == 'b':
  # Upload a file
  is_valid_file = False
  while not is_valid_file:
    is_valid = True
    uploaded = files.upload()
    try:
      file_name = list(uploaded.keys())[0]
      if file_name.split('.')[1] != 'json':
        is_valid = False
      else:
          with open(file_name, "r") as f:
            data_json = json.load(f)
            fields = list(data_json.keys())
            if 'title' not in fields or 'method/part' not in fields:
              is_valid = False
    except:
      is_valid = False
    if is_valid:
      is_valid_file = True
      break
    print('Invalid file, please try again.')
  # Get the commands, descriptions and title
  known_commands, title, descriptions, category_list, title_description = milestone_1_results(file_name, False)

print('Game: ' + title + '\n')
get_description_title(title)
get_description_title_description(title_description)
similars = find_steps_similar_articles(title)
#similars = []
while not end_game:
  if current_step == len(known_commands):
    end_game = True
    print('You reached the end of the game')
  else:
    if current_step == 0:
      print('What do you do first?')
    else:
      print('What do you do next?')
    print('You:')
    #first, second = pick_choices(current_step, known_commands)
    choices = []
    correct_choice = known_commands[current_step] # this is the correct choice
    further_steps = [known_commands[i] for i in range(current_step + 1, len(known_commands))]
    f, s = pick_choices_alt(title, 2, similars, correct_choice, further_steps)
    #f, s = pick_choices(curr_step, known_commands)
    if f:
      choices.append(f)
    if s:
      choices.append(s)
    choices.append(correct_choice)
    random.shuffle(choices) # shuffle the order of choices
    for choice in choices:
      print('\t' + choice.lower())
    correct_found = False
    while not correct_found:
      user_pick = input('>')
      if user_pick.lower() == 'quit':
        correct_found = True
        end_game = True
        print('You ended the game.')
        continue
      # Check if what the user picked is correct
      most_similar = find_similar_command(user_pick, [known_commands[i] for i in range(current_step, len(known_commands))])
      if most_similar.lower() == correct_choice.lower():
        # Correct!
        current_step += 1
        correct_found = True
        # Get description
        print(get_step_description_game(correct_choice, descriptions))
      else:
        print(random.choice(wrong_warning))


Welcome to WikiHow Dungeon!
You can either pick a category to play a game from or you can upload your own game.
a) Pick a category
b) Upload JSON
>b


Saving how_to_safely_hug_your_dog.json to how_to_safely_hug_your_dog.json
Game: How To Safely Hug Your Dog

You want to safely hug your dog
What do you do first?
You:
	give your dog affection
	get the dogs shots, and make sure he\/she has all of them before you go out and about
	give them a warm, petting collar if you want to give them a touch
>give them a warm, petting collar if you want to give them a touch
It’s a good idea to give your dog treats more frequently than every 2 hours or even every hour.
If they don’t want to get too hot or hot, they should simply be petted and held a little, as long as they wait for a pat.
If it’s too hot, you might need to hold your dog for a closer look or give them a longer, petting collar just for them to get used t.

What do you do next?
You:
	have someone photograph you while hugging your dog
	give your dog lots of attention and love
	leave your dog alone
>give your dog lots of attention and love
Umm.. maybe you should not do that now.
>leave you